# Case Study 02 - Method B
## Quantifying the World
## Satvik Ajmera
## Summer 2022

Your case study is to build a classifier using logistic regression to predict hospital readmittance. There is missing data that must be imputed. Once again, discuss variable importances as part of your submission.

In [1]:
import pandas as pd
import numpy as np
from sklearn.preprocessing import OneHotEncoder, StandardScaler
from sklearn.pipeline import Pipeline
from sklearn.compose import ColumnTransformer

In [4]:
diabetes_df = pd.read_csv("../dataset_diabetes/diabetic_data.csv")

In [5]:
ids = pd.read_csv("../dataset_diabetes/IDs_mapping.csv")

In [6]:
ids = ids.dropna(subset=["admission_type_id"])

In [8]:
admission_type_id = ids[0:8]

In [9]:
admission_type_id = admission_type_id.reset_index(drop=True)

In [ ]:
admission_type_id.columns = ["admission_type_id","description"]

In [12]:
discharge_disposition_id = ids[9:39]

In [13]:
discharge_disposition_id.reset_index(inplace=True,drop=True)

In [14]:
discharge_disposition_id.columns = ["discharge_disposition_id","description"]

In [15]:
discharge_disposition_id = discharge_disposition_id.reset_index(drop=True)

In [16]:
discharge_disposition_id["discharge_disposition_id"] = discharge_disposition_id["discharge_disposition_id"].astype(int)

In [17]:
admission_source_id = ids[40:]

In [18]:
admission_source_id.reset_index(inplace=True, drop=True)

In [19]:
admission_source_id.columns = ["admission_source_id", "description"]

In [20]:
admission_source_id["admission_source_id"] = admission_source_id["admission_source_id"].astype(int)

/var/folders/zl/4qqdwn7s6wv34lrr9sxt7tf40000gn/T/ipykernel_9267/1365254308.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  admission_source_id["admission_source_id"] = admission_source_id["admission_source_id"].astype(int)


In [21]:
df = diabetes_df.merge(admission_type_id, on= "admission_type_id", how = "left").merge(discharge_disposition_id, on= "discharge_disposition_id", how = "left").merge(admission_source_id, on= "admission_source_id", how = "left")

In [23]:
df_replaced = df.rename({'description_x': 'admission_type_id_x', 'description_y': 'discharge_disposition_id_y','description':'admission_source_id_z'}, axis=1)

Drop the Subset of all NaN's in the columns of admission_type_id_x 	discharge_disposition_id_y 	admission_source_id_z

In [28]:
df_replaced["patient_nbr"].value_counts()

88785891     40
43140906     28
1660293      23
88227540     23
23199021     23
             ..
11005362      1
98252496      1
1019673       1
13396320      1
175429310     1
Name: patient_nbr, Length: 71518, dtype: int64

In [30]:
pt = df_replaced["patient_nbr"].value_counts().reset_index()

In [32]:
df_drop = df_replaced.drop_duplicates(keep="first",subset="patient_nbr")

In [34]:
df_drop = df_drop.dropna(subset=["admission_type_id_x","discharge_disposition_id_y","admission_source_id_z"])

In [39]:
next_df = df_drop.drop(columns = ["admission_type_id","discharge_disposition_id", "admission_source_id"])

In [40]:
next_df = next_df.replace("?",np.nan)

Drop weight, and payer code for the reasons in the paper. However, replace medical specialty and race missing values to Missing.

In [42]:
next_df = next_df.drop(columns = ["weight","payer_code"])

In [45]:
new_df = next_df.replace(np.nan, "Missing")

Removed hospice and expired for the id columns

In [51]:
removed_df = new_df[(new_df["discharge_disposition_id_y"] != "Hospice / home") & (new_df["discharge_disposition_id_y"] != "Hospice / medical facility") & (new_df["discharge_disposition_id_y"] != "Expired") & (new_df["discharge_disposition_id_y"] != "Expired at home. Medicaid only, hospice.")]

In [53]:
removed_df.reset_index(inplace=True,drop=True)

Finally, we need to drop the encounter_id and patient_nbr

In [55]:
final_df = removed_df.drop(columns = ["encounter_id","patient_nbr"])

In [56]:
final_df.head()

,race,gender,age,time_in_hospital,medical_specialty,num_lab_procedures,num_procedures,num_medications,number_outpatient,number_emergency,number_inpatient,diag_1,diag_2,diag_3,number_diagnoses,max_glu_serum,A1Cresult,metformin,repaglinide,nateglinide,chlorpropamide,glimepiride,acetohexamide,glipizide,glyburide,tolbutamide,pioglitazone,rosiglitazone,acarbose,miglitol,troglitazone,tolazamide,examide,citoglipton,insulin,glyburide-metformin,glipizide-metformin,glimepiride-pioglitazone,metformin-rosiglitazone,metformin-pioglitazone,change,diabetesMed,readmitted,admission_type_id_x,discharge_disposition_id_y,admission_source_id_z
0,Caucasian,Female,[10-20),3,Missing,59,0,18,0,0,0,276,250.01,255,9,None,None,No,No,No,No,No,No,No,No,No,No,No,No,No,No,No,No,No,Up,No,No,No,No,No,Ch,Yes,>30,Emergency,Discharged to home,Emergency Room
1,AfricanAmerican,Female,[20-30),2,Missing,11,5,13,2,0,1,648,250,V27,6,None,None,No,No,No,No,No,No,Steady,No,No,No,No,No,No,No,No,No,No,No,No,No,No,No,No,No,Yes,NO,Emergency,Discharged to home,Emergency Room
2,Caucasian,Male,[30-40),2,Missing,44,1,16,0,0,0,8,250.43,403,7,None,None,No,No,No,No,No,No,No,No,No,No,No,No,No,No,No,No,No,Up,No,No,No,No,No,Ch,Yes,NO,Emergency,Discharged to home,Emergency Room
3,Caucasian,Male,[40-50),1,Missing,51,0,8,0,0,0,197,157,250,5,None,None,No,No,No,No,No,No,Steady,No,No,No,No,No,No,No,No,No,No,Steady,No,No,No,No,No,Ch,Yes,NO,Emergency,Discharged to home,Emergency Room
4,Caucasian,Male,[50-60),3,Missing,31,6,16,0,0,0,414,411,250,9,None,None,No,No,No,No,No,No,No,No,No,No,No,No,No,No,No,No,No,Steady,No,No,No,No,No,No,Yes,>30,Urgent,Discharged to home,Clinic Referral


In [58]:
final_df['readmitted'].value_counts()

NO     36176
>30    19116
<30     5380
Name: readmitted, dtype: int64

Convert NO to class 1, >30 to class 3, <30 to class 2

In [59]:
final_df['readmitted'] = final_df['readmitted'].replace({"NO":1,">30":3,"<30":2})

In [61]:
final_df.head()

,race,gender,age,time_in_hospital,medical_specialty,num_lab_procedures,num_procedures,num_medications,number_outpatient,number_emergency,number_inpatient,diag_1,diag_2,diag_3,number_diagnoses,max_glu_serum,A1Cresult,metformin,repaglinide,nateglinide,chlorpropamide,glimepiride,acetohexamide,glipizide,glyburide,tolbutamide,pioglitazone,rosiglitazone,acarbose,miglitol,troglitazone,tolazamide,examide,citoglipton,insulin,glyburide-metformin,glipizide-metformin,glimepiride-pioglitazone,metformin-rosiglitazone,metformin-pioglitazone,change,diabetesMed,readmitted,admission_type_id_x,discharge_disposition_id_y,admission_source_id_z
0,Caucasian,Female,[10-20),3,Missing,59,0,18,0,0,0,276,250.01,255,9,None,None,No,No,No,No,No,No,No,No,No,No,No,No,No,No,No,No,No,Up,No,No,No,No,No,Ch,Yes,3,Emergency,Discharged to home,Emergency Room
1,AfricanAmerican,Female,[20-30),2,Missing,11,5,13,2,0,1,648,250,V27,6,None,None,No,No,No,No,No,No,Steady,No,No,No,No,No,No,No,No,No,No,No,No,No,No,No,No,No,Yes,1,Emergency,Discharged to home,Emergency Room
2,Caucasian,Male,[30-40),2,Missing,44,1,16,0,0,0,8,250.43,403,7,None,None,No,No,No,No,No,No,No,No,No,No,No,No,No,No,No,No,No,Up,No,No,No,No,No,Ch,Yes,1,Emergency,Discharged to home,Emergency Room
3,Caucasian,Male,[40-50),1,Missing,51,0,8,0,0,0,197,157,250,5,None,None,No,No,No,No,No,No,Steady,No,No,No,No,No,No,No,No,No,No,Steady,No,No,No,No,No,Ch,Yes,1,Emergency,Discharged to home,Emergency Room
4,Caucasian,Male,[50-60),3,Missing,31,6,16,0,0,0,414,411,250,9,None,None,No,No,No,No,No,No,No,No,No,No,No,No,No,No,No,No,No,Steady,No,No,No,No,No,No,Yes,3,Urgent,Discharged to home,Clinic Referral


In [62]:
X = final_df.drop("readmitted", axis = 1).copy()
y = final_df["readmitted"].values.copy()

In [63]:
numeric_features = ['time_in_hospital', 'num_lab_procedures', 'num_procedures',
                    'num_medications','number_outpatient','number_emergency',
                    'number_inpatient','number_diagnoses']

In [64]:
categorical_features = [i for i in X.columns if i not in numeric_features]

## Preprocessing and Standard Scaling use sklearn's pipeline.

In [65]:
numeric_transformer = Pipeline(steps=[('scaler', StandardScaler(with_mean=False))])


categorical_transformer = Pipeline(steps=[('onehot', OneHotEncoder(drop="first"))])


preprocessor = ColumnTransformer(
    transformers=[
        ('num', numeric_transformer, numeric_features),
        ('cat', categorical_transformer, categorical_features)])

pipeline = Pipeline(steps=[('preprocessor', preprocessor)])

In [66]:
X_scaled = pipeline.fit_transform(X)

In [67]:
X_scaled

<60672x2310 sparse matrix of type '<class 'numpy.float64'>'
	with 1764138 stored elements in Compressed Sparse Row format>

# Logistic Regression with no regularization

In [71]:
from sklearn.model_selection import StratifiedShuffleSplit 
sss_cv = StratifiedShuffleSplit(n_splits=5,test_size=0.10, random_state=42)

In [72]:
%%time
from sklearn.linear_model import LogisticRegression
from sklearn import metrics as mt
import time

lr_clf = LogisticRegression(multi_class = "multinomial",max_iter=5000, random_state=42)
acc_scores = []
iter_num=0

for train_indices, test_indices in sss_cv.split(X_scaled,y): 
    X_train = X_scaled[train_indices]
    y_train = y[train_indices]
    
    X_test = X_scaled[test_indices]
    y_test = y[test_indices]
    lr_clf.fit(X_train,y_train)  # train object

    y_hat = lr_clf.predict(X_test) # get test set predictions
    print("====Iteration",iter_num," ====")
    acc = mt.accuracy_score(y_test,y_hat)
    acc_scores.append(acc)
    print('Accuracy:', acc)
    iter_num+=1

====Iteration 0  ====
Accuracy: 0.6143704680290046
====Iteration 1  ====
Accuracy: 0.6058009228740936
====Iteration 2  ====
Accuracy: 0.6092617007251153
====Iteration 3  ====
Accuracy: 0.6086025049439684
====Iteration 4  ====
Accuracy: 0.6072841133816743
CPU times: user 2min 4s, sys: 4.37 s, total: 2min 8s
Wall time: 2min 8s


In [73]:
print("Mean Accuracy is:",np.mean(acc))

Mean Accuracy is: 0.6072841133816743


# L1 Regularization Grid Search

In [74]:
from sklearn.model_selection import StratifiedShuffleSplit 
sss_cv = StratifiedShuffleSplit(n_splits=5,test_size=0.10, random_state=42)

In [75]:
%%time
from sklearn.linear_model import LogisticRegression
from sklearn.model_selection import GridSearchCV


regEstimator = LogisticRegression(multi_class = "multinomial")

parameters = { 'penalty':['l1']
              ,'C': [0.001, 0.01, 0.1]
              ,'random_state': [42]
              ,'solver': ['saga']
              ,'max_iter':[5000]
             }


GridSearch = GridSearchCV(estimator=regEstimator
                   , verbose=3
                   , param_grid=parameters
                   , cv=sss_cv 
                   , scoring='accuracy')

GridSearch.fit(X_scaled, y)

Fitting 5 folds for each of 3 candidates, totalling 15 fits
[CV 1/5] END C=0.001, max_iter=5000, penalty=l1, random_state=42, solver=saga;, score=0.602 total time=  46.0s
[CV 2/5] END C=0.001, max_iter=5000, penalty=l1, random_state=42, solver=saga;, score=0.603 total time=  45.8s
[CV 3/5] END C=0.001, max_iter=5000, penalty=l1, random_state=42, solver=saga;, score=0.602 total time=  45.0s
[CV 4/5] END C=0.001, max_iter=5000, penalty=l1, random_state=42, solver=saga;, score=0.601 total time=  45.7s
[CV 5/5] END C=0.001, max_iter=5000, penalty=l1, random_state=42, solver=saga;, score=0.602 total time=  47.1s
[CV 1/5] END C=0.01, max_iter=5000, penalty=l1, random_state=42, solver=saga;, score=0.608 total time= 2.6min
[CV 2/5] END C=0.01, max_iter=5000, penalty=l1, random_state=42, solver=saga;, score=0.608 total time= 2.4min
[CV 3/5] END C=0.01, max_iter=5000, penalty=l1, random_state=42, solver=saga;, score=0.610 total time= 2.7min
[CV 4/5] END C=0.01, max_iter=5000, penalty=l1, random_

GridSearchCV(cv=StratifiedShuffleSplit(n_splits=5, random_state=42, test_size=0.1,
            train_size=None),
             estimator=LogisticRegression(multi_class='multinomial'),
             param_grid={'C': [0.001, 0.01, 0.1], 'max_iter': [5000],
                         'penalty': ['l1'], 'random_state': [42],
                         'solver': ['saga']},
             scoring='accuracy', verbose=3)

In [76]:
l1_best_est = GridSearch.best_estimator_

In [77]:
print(l1_best_est)

LogisticRegression(C=0.1, max_iter=5000, multi_class='multinomial',
                   penalty='l1', random_state=42, solver='saga')


In [78]:
l1_best_params = GridSearch.best_params_

In [79]:
print(l1_best_params)

{'C': 0.1, 'max_iter': 5000, 'penalty': 'l1', 'random_state': 42, 'solver': 'saga'}


In [80]:
l1_best_score = GridSearch.best_score_

In [81]:
print('Highest Accuracy', l1_best_score )

Highest Accuracy 0.6132498352010547


# L2 Regularization Gridsearch

In [82]:
from sklearn.model_selection import StratifiedShuffleSplit 
sss_cv = StratifiedShuffleSplit(n_splits=5,test_size=0.10, random_state=42)

In [83]:
%%time
from sklearn.linear_model import LogisticRegression
from sklearn.model_selection import GridSearchCV


regEstimator = LogisticRegression(multi_class = "multinomial")

parameters = { 'penalty':['l2']
              ,'C': [0.001, 0.01, 0.1]
              ,'random_state': [42]
              ,'solver': ['saga']
              ,'max_iter':[5000]
             }


l2_GridSearch = GridSearchCV(estimator=regEstimator
                   , verbose=3
                   , param_grid=parameters
                   , cv=sss_cv 
                   , scoring='accuracy')

l2_GridSearch.fit(X_scaled, y)

Fitting 5 folds for each of 3 candidates, totalling 15 fits
[CV 1/5] END C=0.001, max_iter=5000, penalty=l2, random_state=42, solver=saga;, score=0.609 total time=   0.7s
[CV 2/5] END C=0.001, max_iter=5000, penalty=l2, random_state=42, solver=saga;, score=0.609 total time=   0.7s
[CV 3/5] END C=0.001, max_iter=5000, penalty=l2, random_state=42, solver=saga;, score=0.612 total time=   0.7s
[CV 4/5] END C=0.001, max_iter=5000, penalty=l2, random_state=42, solver=saga;, score=0.611 total time=   0.7s
[CV 5/5] END C=0.001, max_iter=5000, penalty=l2, random_state=42, solver=saga;, score=0.606 total time=   0.7s
[CV 1/5] END C=0.01, max_iter=5000, penalty=l2, random_state=42, solver=saga;, score=0.615 total time=   3.9s
[CV 2/5] END C=0.01, max_iter=5000, penalty=l2, random_state=42, solver=saga;, score=0.609 total time=   3.9s
[CV 3/5] END C=0.01, max_iter=5000, penalty=l2, random_state=42, solver=saga;, score=0.613 total time=   4.3s
[CV 4/5] END C=0.01, max_iter=5000, penalty=l2, random_

GridSearchCV(cv=StratifiedShuffleSplit(n_splits=5, random_state=42, test_size=0.1,
            train_size=None),
             estimator=LogisticRegression(multi_class='multinomial'),
             param_grid={'C': [0.001, 0.01, 0.1], 'max_iter': [5000],
                         'penalty': ['l2'], 'random_state': [42],
                         'solver': ['saga']},
             scoring='accuracy', verbose=3)

In [84]:
l2_best_est = l2_GridSearch.best_estimator_

In [85]:
print(l2_best_est)

LogisticRegression(C=0.1, max_iter=5000, multi_class='multinomial',
                   random_state=42, solver='saga')


In [86]:
l2_best_params = l2_GridSearch.best_params_

In [87]:
print(l2_best_params)

{'C': 0.1, 'max_iter': 5000, 'penalty': 'l2', 'random_state': 42, 'solver': 'saga'}


In [88]:
l2_best_score = l2_GridSearch.best_score_

In [89]:
print('Highest Accuracy', l2_best_score )

Highest Accuracy 0.6141067897165458


# Variable Importance - L2 Regularization

In [90]:
numeric_features_new = ['time_in_hospital', 'num_lab_procedures', 'num_procedures',
                    'num_medications','number_outpatient','number_emergency',
                    'number_inpatient','number_diagnoses']

X_columns = final_df.drop("readmitted", axis = 1).copy()
X_columns = pipeline.fit(X_columns)
X_columns.named_steps["preprocessor"].transformers_[1][1].named_steps["onehot"]

column_name = X_columns.named_steps['preprocessor'].transformers_[1][1]\
   .named_steps['onehot'].get_feature_names_out(categorical_features)

column_name = list(column_name)
column_name[:0] = numeric_features_new

In [91]:
l2_coefficients = pd.concat([pd.DataFrame(column_name),pd.DataFrame(np.transpose(l2_best_est.coef_))], axis = 1)

In [92]:
l2_coefficients.columns = ["Features", "NO", "<30", ">30"]

In [93]:
l2_coefficients['Absolute_Sum'] = abs(l2_coefficients["NO"]) + abs(l2_coefficients["<30"]) + abs(l2_coefficients[">30"])

In [95]:
 sort_coef_l2 = l2_coefficients.sort_values(by="Absolute_Sum", ascending=False)

In [97]:
sort_coef_l2.head(20)

,Features,NO,<30,>30,Absolute_Sum
772,diag_1_V58,-0.546077,0.828365,-0.282288,1.656730
2283,discharge_disposition_id_y_Discharged/transfer...,0.419779,-0.643759,0.223980,1.287519
2291,discharge_disposition_id_y_Discharged/transfer...,-0.270346,0.619027,-0.348680,1.238054
2290,discharge_disposition_id_y_Discharged/transfer...,-0.477164,0.553386,-0.076222,1.106772
2285,discharge_disposition_id_y_Discharged/transfer...,-0.315836,0.525705,-0.209869,1.051410
853,diag_2_250.03,-0.517599,0.359076,0.158523,1.035199
371,diag_1_443,-0.490329,0.408563,0.081766,0.980658
1818,diag_3_536,-0.479923,0.343131,0.136792,0.959846
2279,discharge_disposition_id_y_Discharged to home,0.156270,-0.470239,0.313969,0.940478
864,diag_2_250.41,-0.438645,0.241545,0.197100,0.877290


In [98]:
sort_coef_l2.reset_index(inplace=True,drop=True)

In [100]:
sort_coef_l2.to_csv("method_2_variable_imp.csv",index=False)